In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [5]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [8]:
import keras
from keras import utils
from keras.layers import Layer
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, CSVLogger
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import pickle
from scipy.signal import savgol_filter
from sklearn import metrics
from sklearn.model_selection import train_test_split
from skimage import exposure
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
import glob
import matplotlib.pyplot as plt

%matplotlib inline

--------------------------

In [14]:
keras.backend.clear_session()

model = ResNet50(include_top=True,
                 weights=None,
                 input_tensor=None,
                 input_shape=(512,512,6),
                 pooling=None,
                 classes=50)

model.compile(loss=focal_loss,
              optimizer=Adam(0.0001),
              metrics=['categorical_accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 6)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 6)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 64) 18880       conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 256, 256, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [18]:
base_path = '../../data/train_parsed_first_50_hist_matched/'
x_id, y_id = np.load("../../data/x_id_first_50_hist_matched.npy"), np.load("../../data/y_id_first_50_hist_matched.npy")

classes = [np.argmax(np.load(base_path+id+".npy")) for id in y_id]

In [19]:
training_id, val_id, _, _ = train_test_split(x_id, classes, test_size=0.15, random_state=42)

num_training_samples = len(training_id)
num_val_samples = len(val_id)

In [20]:
print(num_training_samples)
print(num_val_samples)

2803
495


In [23]:
# Parameters
params = {'dim': (512,512),
          'batch_size': 4,
          'n_classes': 50,
          'n_channels': 6,
          'shuffle': True}

id_dict = {'train': training_id,
           'val': val_id}

# Generators
training_generator = DataGenerator(id_dict['train'], **params)
validation_generator = DataGenerator(id_dict['val'], **params)

In [24]:
logger = CSVLogger('../logs/training_resnet50_072719.log', 
                   separator=',', 
                   append=False)

In [25]:
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', 
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              mode='max',
                              cooldown=0,
                              min_lr=0.00001)

In [ ]:
# Train model on dataset
model.fit_generator(generator=training_generator,
                    steps_per_epoch=np.ceil(num_training_samples / params['batch_size']),
                    epochs=25,
                    verbose=1,
                    callbacks=[logger, reduce_lr],
                    validation_data=validation_generator,
                    validation_steps=np.ceil(num_val_samples / params['batch_size']),
                    class_weight=None,
                    max_queue_size=20,
                    workers=6,
                    use_multiprocessing=False,
                    shuffle=True,
                    initial_epoch=0)

Epoch 1/25
701/701 [==============================] - 300s 429ms/step - loss: 4.0601 - categorical_accuracy: 0.0185 - val_loss: 4.2016 - val_categorical_accuracy: 0.0343
Epoch 2/25
701/701 [==============================] - 279s 399ms/step - loss: 3.7775 - categorical_accuracy: 0.0303 - val_loss: 4.1879 - val_categorical_accuracy: 0.0282
Epoch 3/25
701/701 [==============================] - 271s 387ms/step - loss: 3.7249 - categorical_accuracy: 0.0310 - val_loss: 4.4404 - val_categorical_accuracy: 0.0343
Epoch 4/25
701/701 [==============================] - 295s 420ms/step - loss: 3.7046 - categorical_accuracy: 0.0321 - val_loss: 3.9227 - val_categorical_accuracy: 0.0202

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
Epoch 5/25
701/701 [==============================] - 278s 396ms/step - loss: 3.5876 - categorical_accuracy: 0.0471 - val_loss: 3.7684 - val_categorical_accuracy: 0.0323
Epoch 6/25
701/701 [==============================] - 268s 382ms/ste

In [ ]:
#acc = 0.35 by epoch 10
#val_acc = 0.3 be epoch 10

### Model/function definitions

In [9]:
def f1_loss(y_true, y_pred):
    #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

In [10]:
gamma = 2.0
epsilon = K.epsilon()

def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    return K.sum(FL, axis=1)

In [11]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('../../data/train_parsed_first_50_hist_matched/' + ID + '.npy')

            # Store class
            y[i,] = np.load('../../data/train_parsed_first_50_hist_matched/y' + ID[1:] + '.npy')    

        return X, y
    
    def __equalize(self, x):
        'Contrast stretching'
        p2, p98 = np.percentile(x, (2, 98))
        return exposure.rescale_intensity(x, in_range=(p2, p98))

## Archive

In [ ]:
# model = InceptionResNetV2(include_top=True,
#                           weights=None,
#                           input_tensor=None,
#                           input_shape=(512,512,6),
#                           pooling=None,
#                           classes=50)